<a href="https://colab.research.google.com/github/hi13i/DLNoteClass/blob/master/DLNoteClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install --pre torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install torchinfo
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary
from sklearn.metrics import classification_report
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os

torch.manual_seed(17)

if not torch.cuda.is_available():
  print('\nGPU が割り当てられていません.')
  print('「編集」>「ノートブックの設定」から「ハードウェア アクセラレータ」を GPU に変更して再実行してください.')
  print('なお「GPU バックエンドに接続できません」と表示される場合にはしばらく経過してから再実行してください.')
else:
  print('\nGPU が利用できます.')
  print('無料版では GPU として Tesla K80 か Tesla T4 が割り当てられます.')
  print('Tesla T4 の方が高性能で学習も速いですが、何度も利用しているユーザには Tesla K80 が割り当てられることが多いようです.\n')
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/nightly/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

GPU が利用できます.
無料版では GPU として Tesla K80 か Tesla T4 が割り当てられます.
Tesla T4 の方が高性能で学習も速いですが、何度も利用しているユーザには Tesla K80 が割り当てられることが多いようです.

Mon Jun 12 00:52:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 

In [2]:
!git clone https://github.com/hi13i/DLNoteClass.git


Cloning into 'DLNoteClass'...
fatal: could not read Username for 'https://github.com': No such device or address
